In [1]:
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import sys
sys.path.append("/code")

from tqdm import tqdm
import torch
# device = torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
import gym
import recogym

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

from sklearn.utils import check_random_state

# implementing OPE of the IPWLearner using synthetic bandit data
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

from scipy.special import softmax
import optuna


from estimators import (
    DirectMethod as DM
)

from simulation_utils import (
    eval_policy,
    generate_dataset,
    create_simulation_data_from_pi,
    get_train_data,
    get_opl_results_dict,
    CustomCFDataset,
    calc_reward
)

from models import (    
    CFModel,
    NeighborhoodModel,
    BPRModel
)

from training_utils import (
    fit_bpr,
    train,
    validation_loop
 )

from custom_losses import (
    SNDRPolicyLoss,
    BPRLoss
    )

random_state=12345
random_ = check_random_state(random_state)

Using device: cpu
Using device: cpu
Using device: cpu


In [2]:
pd.options.display.float_format = '{:,.4f}'.format

## `trainer_trial` Function

This function runs policy learning experiments using offline bandit data and evaluates various estimators.

### Parameters
- **num_runs** (int): Number of experimental runs per training size
- **num_neighbors** (int): Number of neighbors to consider in the neighborhood model
- **num_rounds_list** (list): List of training set sizes to evaluate
- **dataset** (dict): Contains dataset information including embeddings, action probabilities, and reward probabilities
- **batch_size** (int): Batch size for training the policy model
- **num_epochs** (int): Number of training epochs for each experiment
- **lr** (float, default=0.001): Learning rate for the optimizer

### Process Flow
1. Initializes result structures and retrieval models
2. For each training size in `num_rounds_list`:
   - Creates a uniform logging policy and simulates data
   - Generates training data for offline learning
   - Fits regression and neighborhood models for reward estimation
   - Initializes and trains a counterfactual policy model
   - Evaluates policy performance using various estimators
   - Collects metrics on policy reward and embedding quality

### Returns
- **DataFrame**: Results table with rows indexed by training size and columns for various metrics:
  - `policy_rewards`: True expected reward of the learned policy
  - Various estimator errors (`ipw`, `reg_dm`, `conv_dm`, `conv_dr`, `conv_sndr`)
  - Variance metrics for each estimator
  - Embedding quality metrics comparing learned representations to ground truth

### Implementation Notes
- Uses uniform random logging policy for collecting offline data
- Employs Self-Normalized Doubly Robust (SNDR) policy learning
- Measures embedding quality via RMSE to original/ground truth embeddings

In [3]:
def trainer_trial(
                  num_runs,
                  num_neighbors,
                  num_rounds_list,
                  dataset,
                  batch_size
                  ):
    
    # Define device at the beginning
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    def objective(trial):
    
        # Optuna objective function
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)
        epochs = trial.suggest_int("num_epochs", 1, 10)

        trial_neigh_model = NeighborhoodModel(
                                                train_data['x_idx'],
                                                train_data['a'], 
                                                our_a,
                                                our_x, 
                                                train_data['r'], 
                                                num_neighbors=num_neighbors
                                            )
        

        trial_model = CFModel(
                        n_users, 
                        n_actions, 
                        emb_dim, 
                        initial_user_embeddings=torch.tensor(our_x, device=device), 
                        initial_actions_embeddings=torch.tensor(our_a, device=device)
                        )
        
        # Training
        train(trial_model, train_loader, trial_neigh_model, criterion=SNDRPolicyLoss(), num_epochs=epochs, lr=lr, device=device)
        return validation_loop(trial_model, val_loader, trial_neigh_model)
    
    device = torch.device('cpu')

    dm = DM()
    results = {}

    our_x, our_a = dataset["our_x"], dataset["our_a"]
    emb_x, emb_a = dataset["emb_x"], dataset["emb_a"]
    original_x, original_a = dataset["original_x"], dataset["original_a"]
    n_users, n_actions, emb_dim = dataset["n_users"], dataset["n_actions"], dataset["emb_dim"]
    
    first = True

    for train_size in num_rounds_list:
        reg_results, conv_results = [], []
        
        for run in range(num_runs):

            pi_0 = softmax(our_a @ our_x.T, axis=1)
            original_policy_prob = np.expand_dims(pi_0, -1)
            simulation_data = create_simulation_data_from_pi(
                                                            dataset['env'],
                                                            pi_0,
                                                            train_size + 500
                                                            )
            # test_data = get_test_data(dataset, simulation_data, n_test_data)

            # idx = np.arange(train_size) + n_test_data
            idx = np.arange(train_size)

            train_data = get_train_data(n_actions, train_size, simulation_data, idx, our_x)
            val_data = get_train_data(n_actions, 500, simulation_data, np.arange(500) + train_size, our_x)

            # bpr_model = BPRModel(
            #                     n_users,
            #                     n_actions,
            #                     emb_x.shape[1], 
            #                     initial_user_embeddings=torch.tensor(our_x, device=device), 
            #                     initial_actions_embeddings=torch.tensor(our_a, device=device)
            #                     )
            
            neighberhoodmodel = NeighborhoodModel(
                                                    train_data['x_idx'],
                                                    train_data['a'], 
                                                    our_a,
                                                    our_x, 
                                                    train_data['r'], 
                                                    num_neighbors=num_neighbors
                                                )
            

            model = CFModel(
                            n_users, 
                            n_actions, 
                            emb_dim, 
                            initial_user_embeddings=torch.tensor(our_x, device=device), 
                            initial_actions_embeddings=torch.tensor(our_a, device=device)
                            )
            
            cf_dataset =  CustomCFDataset(
                                       train_data['x_idx'], 
                                       train_data['a'], 
                                       train_data['r'], 
                                       original_policy_prob[train_data['x_idx']]
                                       )
            
            train_loader = DataLoader(cf_dataset, batch_size=batch_size, shuffle=True)

            val_dataset =  CustomCFDataset(
                            val_data['x_idx'], 
                            val_data['a'], 
                            val_data['r'], 
                            original_policy_prob[val_data['x_idx']]
                            )
            
            val_loader = DataLoader(val_dataset, batch_size=len(val_data['r']), shuffle=True)
            
            if first:
                policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)
                conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob[train_data['x_idx']], policy))
                conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])
                conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])
                # bpr_scores = bpr_model.calc_scores(torch.tensor(train_data['x_idx'], device=device, dtype=torch.long)).detach().cpu().numpy()
                # reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], bpr_scores)
                reg_dm = 0.0
                reg_results.append(reg_dm)
                first = False
                reg_results = np.array(reg_results)
                conv_results = np.array(conv_results)
                results[0] = get_opl_results_dict(reg_results, conv_results)
                reg_results, conv_results = [], []
            
            # Bloss = BPRLoss()

            
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=10)
            best_params = study.best_params

            neighberhoodmodel = NeighborhoodModel(
                                                    train_data['x_idx'],
                                                    train_data['a'], 
                                                    our_a,
                                                    our_x, 
                                                    train_data['r'], 
                                                    num_neighbors=num_neighbors
                                                )
            

            model = CFModel(
                            n_users, 
                            n_actions, 
                            emb_dim, 
                            initial_user_embeddings=torch.tensor(our_x, device=device), 
                            initial_actions_embeddings=torch.tensor(our_a, device=device)
                            )

            train(model, train_loader, neighberhoodmodel, criterion=SNDRPolicyLoss(), num_epochs=best_params['num_epochs'], lr=best_params['lr'], device=device)
            # fit_bpr(bpr_model, Bloss, train_loader, num_epochs=3, lr=0.001, device=device)
            # neighborhood_model.update(model.get_params()[0].detach().numpy(), model.get_params()[1].detach().numpy())'

            our_a, our_x = model.get_params()
            our_a, our_x = our_a.detach().cpu().numpy(), our_x.detach().cpu().numpy()


            policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)

            # bpr_scores = bpr_model.calc_scores(torch.tensor(train_data['x_idx'], device=device, dtype=torch.long)).detach().cpu().numpy()
            # reg_dm = dm.estimate_policy_value(policy[train_data['x_idx']], bpr_scores)
            reg_dm = 0.0

            reg_results.append(reg_dm)

            # conv_results.append(eval_policy(neighberhoodmodel, test_data, original_policy_prob[test_data['x_idx']], policy))
            conv_results.append(eval_policy(neighberhoodmodel, train_data, original_policy_prob[train_data['x_idx']], policy))

            conv_results[-1] = np.append(calc_reward(dataset, policy), conv_results[-1])
            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_a-our_a)**2)), np.sqrt(np.mean((original_a-our_a)**2))])

            # temp.append(np.mean((emb_a-our_a)**2, axis=0))

            conv_results[-1] = np.append(conv_results[-1], [np.sqrt(np.mean((emb_x-our_x)**2)), np.sqrt(np.mean((original_x-our_x)**2))])
            
            our_a, our_x = original_a.copy(), original_x.copy()

        reg_results = np.array(reg_results)
        conv_results = np.array(conv_results)

        results[train_size] = get_opl_results_dict(reg_results, conv_results)
    
    return pd.DataFrame.from_dict(results, orient='index')

## Learning

We will run several simulations on a generated dataset, the dataset is generated like this:
$$ \text{We have users U and actions A } u_i \sim N(0, I_{emb_dim}) \ a_i \sim N(0, I_{emb_dim})$$
$$ p_{ij} = 1 / (5 + e^{-(u_i.T a_j)}) $$
$$r_{ij} \sim Bin(p_{ij})$$

We have a policy $\pi$
and it's ground truth reward is calculated by
$$R_{gt} = \sum_{i}{\sum_{j}{\pi_{ij} * p_{ij}}} $$

Our parameters for the dataset will be
$$EmbDim = 5$$
$$NumActions= 150$$
$$NumUsers = 150$$
$$NeighborhoodSize = 6$$

to learn a new policy from $\pi$ we will sample from:
$$\pi_{start} = (1-\epsilon)*\pi + \epsilon * \pi_{random}$$

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [5]:
num_runs = 1

In [6]:
dataset_params = dict(
                    n_actions= 150,
                    n_users = 150,
                    emb_dim = 5,
                    # sigma = 0.1,
                    eps = 0.3 # this is the epsilon for the noise in the ground truth policy representation
                    )

train_dataset = generate_dataset(dataset_params)

In [7]:
num_runs = 1
batch_size = 50
num_neighbors = 6
num_rounds_list = [200, 400, 600, 800, 1000, 2000, 4000]

### 1

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.005$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [8]:
df4 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size)

[I 2025-07-10 06:45:32,122] A new study created in memory with name: no-name-a514921e-f2f0-403d-88f4-a75a5ef76398
[I 2025-07-10 06:45:32,367] Trial 0 finished with value: -0.052747921026163345 and parameters: {'lr': 0.05802445000559664, 'num_epochs': 9}. Best is trial 0 with value: -0.052747921026163345.
[I 2025-07-10 06:45:32,588] Trial 1 finished with value: -0.02691320859602407 and parameters: {'lr': 0.0015417202355803839, 'num_epochs': 7}. Best is trial 0 with value: -0.052747921026163345.
[I 2025-07-10 06:45:32,824] Trial 2 finished with value: -0.049922192406169394 and parameters: {'lr': 0.04359832746188537, 'num_epochs': 10}. Best is trial 0 with value: -0.052747921026163345.
[I 2025-07-10 06:45:33,036] Trial 3 finished with value: -0.026482111012396767 and parameters: {'lr': 0.00018352484549730537, 'num_epochs': 5}. Best is trial 0 with value: -0.052747921026163345.
[I 2025-07-10 06:45:33,275] Trial 4 finished with value: -0.026488384816238116 and parameters: {'lr': 0.000157618

ValueError: `action_dist` must be a probability distribution

In [ ]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.1815,0.0035,0.1541,0.0128,0.0317,0.0333,0.3386,0.0000,0.5364,0.0000
1,0.1904,0.1899,0.1524,0.0293,0.1332,0.5069,1.0378,1.1542,0.8883,0.6694
2,0.1913,0.1715,0.1413,0.0089,0.0080,0.1411,1.1592,1.2819,1.0435,0.8357
3,0.1934,0.1934,0.1476,0.0063,0.0059,0.2330,1.3957,1.5383,1.2811,1.1333
4,0.1916,0.1916,0.1409,0.0330,0.1543,0.4980,1.5903,1.7414,1.5547,1.3738
5,0.1923,0.1923,0.1428,0.0013,0.1317,0.3333,1.8113,1.9436,1.9940,1.8124
10,0.1890,0.0114,0.1402,0.0054,0.0385,0.0718,2.1439,2.2478,2.7655,2.5865
20,0.1889,0.0217,0.1330,0.0297,0.0198,0.0187,2.1254,2.2207,2.8812,2.6593


In [ ]:
num_rounds_list = [1]

### 2

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.001$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [ ]:
df5 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size)

[I 2025-06-26 06:00:47,088] A new study created in memory with name: no-name-f919595e-a7c7-4192-82a0-88df61c0332c
[I 2025-06-26 06:00:47,250] Trial 0 finished with value: 0.037067933837222816 and parameters: {'lr': 0.0005225653346301143, 'num_epochs': 6}. Best is trial 0 with value: 0.037067933837222816.
[I 2025-06-26 06:00:47,387] Trial 1 finished with value: 0.037253001270894676 and parameters: {'lr': 0.00027374423899567256, 'num_epochs': 1}. Best is trial 0 with value: 0.037067933837222816.
[I 2025-06-26 06:00:47,560] Trial 2 finished with value: -0.014201574651007048 and parameters: {'lr': 0.03807918977276577, 'num_epochs': 9}. Best is trial 2 with value: -0.014201574651007048.
[I 2025-06-26 06:00:47,701] Trial 3 finished with value: 0.03726279086963771 and parameters: {'lr': 5.0536893389243686e-05, 'num_epochs': 2}. Best is trial 2 with value: -0.014201574651007048.
[I 2025-06-26 06:00:47,859] Trial 4 finished with value: 0.037096458943192856 and parameters: {'lr': 0.0005744307172

In [ ]:
df5

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.1815,0.0035,0.1541,0.0128,0.0317,0.0333,0.0000,0.0000,0.0000,0.0000,0.0000,0.3386,0.0000,0.5364,0.0000
1,0.1448,0.2204,0.1381,0.0131,0.0874,0.1590,0.0000,0.0000,0.0000,0.0000,0.0000,1.4769,1.3154,1.5475,1.3733


### 3

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.003$$
$$n_{epochs} = 10$$
$$BatchSize=50$$

In [ ]:
df6 = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size)

TypeError: trainer_trial() got an unexpected keyword argument 'num_epochs'

In [ ]:
df6

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.1815,0.0035,0.1541,0.0128,0.0317,0.0333,0.0000,0.0000,0.0000,0.0000,0.0000,0.3386,0.0000,0.5364,0.0000
1,0.1451,0.0462,0.1381,0.0028,0.0221,0.0240,0.0000,0.0000,0.0000,0.0000,0.0000,1.2730,1.0816,1.2032,1.0708
2,0.1453,0.0158,0.1383,0.0246,0.0427,0.0336,0.0000,0.0000,0.0000,0.0000,0.0000,1.2871,1.0995,1.2186,1.0893
3,0.1453,0.0045,0.1382,0.0137,0.0115,0.0118,0.0000,0.0000,0.0000,0.0000,0.0000,1.3095,1.1251,1.2390,1.1124
4,0.1453,0.0006,0.1383,0.0117,0.0018,0.0006,0.0000,0.0000,0.0000,0.0000,0.0000,1.3261,1.1443,1.2600,1.1368
5,0.1449,0.0336,0.1377,0.0085,0.0359,0.0393,0.0000,0.0000,0.0000,0.0000,0.0000,1.3455,1.1664,1.2692,1.1500
10,0.1448,0.0236,0.1370,0.0037,0.0011,0.0013,0.0000,0.0000,0.0000,0.0000,0.0000,1.4985,1.3407,1.3654,1.2810
20,0.1476,0.0106,0.1411,0.0031,0.0055,0.0050,0.0000,0.0000,0.0000,0.0000,0.0000,2.0338,1.9246,1.6279,1.6433


### 4

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.05$$
$$n_{epochs} = 10$$
$$BatchSize=150$$

In [ ]:
df7 = trainer_trial(num_runs, num_neighbors, num_rounds_list[:-3], train_dataset, batch_size+100, num_epochs=10, lr=0.05)

Epoch [10/10], Loss: 85.7448: 100%|██████████| 10/10 [00:00<00:00, 25.09it/s]


In [ ]:
df7

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,ipw_var,reg_dm_var,conv_dm_var,conv_dr_var,conv_sndr_var,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.1815,0.0035,0.1541,0.0128,0.0317,0.0333,0.0000,0.0000,0.0000,0.0000,0.0000,0.3386,0.0000,0.5364,0.0000
1,0.1430,0.0497,0.1361,0.0009,0.0148,0.0294,0.0000,0.0000,0.0000,0.0000,0.0000,1.4746,1.3141,1.3637,1.2353
2,0.1449,0.1010,0.1397,0.0187,0.1998,0.1172,0.0000,0.0000,0.0000,0.0000,0.0000,1.7363,1.6121,1.5940,1.5158
3,0.1450,0.1332,0.1399,0.0072,0.1406,0.1160,0.0000,0.0000,0.0000,0.0000,0.0000,2.1355,2.0325,1.8826,1.8401
4,0.1486,0.0274,0.1391,0.0125,0.0855,0.1512,0.0000,0.0000,0.0000,0.0000,0.0000,2.5199,2.4330,2.1781,2.1835
